In [ ]:
!pip install deap
import random
from deap import base, creator, tools, algorithms
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt


# Definir el tipo de problema (maximización)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Datos de plataformas y contenido
plataformas = [
    {"nombre": "Netflix", "precio": 10, "contenido": ["Better Call Saul", "Black Mirror", "Money Heist", "Narcos", "The Witcher", "Ozark", "The Umbrella Academy", "BoJack Horseman", "Lucifer", "The Haunting of Hill House", "Dark", "Peaky Blinders", "Friends", "Breaking Bad", "Vikings", "The Queen's Gambit", "The Irishman", "Pulp Fiction", "The Matrix"]},
    {"nombre": "Disney Plus", "precio": 9, "contenido": ["The Mandalorian", "Ironman", "The Falcon and the Winter Soldier", "Soul", "Raya and the Last Dragon", "Hamilton", "Frozen", "Moana", "Aladdin", "The Lion King", "Zootopia", "The Simpsons", "Toy Story", "Beauty and the Beast", "Marvel's Avengers", "Guardians of the Galaxy", "Doctor Strange", "Pirates of the Caribbean", "The Incredibles"]},
    {"nombre": "Primevideo", "precio": 8, "contenido": ["The Boys", "The Marvelous Mrs. Maisel", "The Expanse", "Invincible", "Fleabag", "Hunters", "Tom Clancy's Jack Ryan", "The Grand Tour", "The Man in the High Castle", "Vikings", "The Office", "Downtown Abbey", "Borat Subsequent Moviefilm", "Sound of Metal", "One Night in Miami", "Palm Springs", "The Big Sick", "Mission: Impossible - Fallout", "The Sound of Music", "Gladiator"]},
    {"nombre": "HBO Max", "precio": 12, "contenido": ["Game of Thrones", "Westworld", "Succession", "Watchmen", "Euphoria", "His Dark Materials", "Chernobyl", "Friends: The Reunion", "The Sopranos", "The Wire", "Insecure", "Lovecraft Country", "Perry Mason", "The Undoing", "The Little Things", "Dune", "The Matrix Resurrections", "Wonder Woman 1984", "Joker", "Harry Potter"]},
    {"nombre": "Apple TV Plus", "precio": 7, "contenido": ["Ted Lasso", "The Morning Show", "Defending Jacob", "Dickinson", "Servant", "For All Mankind", "Mythic Quest: Raven's Banquet", "Central Park", "Greyhound", "Palmer", "Wolfwalkers", "Billie Eilish: The World's a Little Blurry", "Soul", "The Banker", "On the Rocks", "No Time to Die", "Cherry", "Greyhound", "Ted Lasso", "CODA"]},
    {"nombre": "Paramount Plus", "precio": 8, "contenido": ["Star Trek: Discovery", "The Good Fight", "Twilight Zone", "SpongeBob SquarePants", "iCarly", "Dora the Explorer", "Mission Impossible 7", "Top Gun: Maverick", "The Stand", "The Haunting of Bly Manor", "Paw Patrol", "Blue's Clues", "Scream", "A Quiet Place Part II", "Avatar: The Last Airbender", "Indiana Jones 5", "South Park", "Young Sheldon", "Criminal Minds", "Ray Donovan"]},
    {"nombre": "Star+", "precio": 5, "contenido": ["The Walking Dead", "Los Simpson", "This Is Us", "Anatomía según Grey", "Prison Break", "Perdidos", "Scandal", "Padre de familia", "Modern Family", "The Gifted", "24", "How I Met Your Mother", "Expediente X", "The Resident", "The Orville", "Feud: Bette and Joan", "Big Sky", "Pam & Tommy", "Futurama", "Bob's Burgers"]},
    {"nombre": "Peacock", "precio": 3, "contenido": ["The Office", "Parks and Recreation", "Yellowstone", "The Blacklist", "Brooklyn Nine-Nine", "Law & Order: SVU", "Mr. Robot", "Dexter: New Blood", "Saved by the Bell", "Brave New World", "Dr. Death", "Girls5Eva", "Psych 2: Lassie Come Home", "Fast & Furious 9", "Jurassic World: Dominion", "Trolls World Tour", "The Croods: A New Age", "Boss Baby: Family Business", "Harry Potter and the Sorcerer's Stone", "Transformers"]},
    {"nombre": "Crunchyroll", "precio": 2, "contenido": ["One Piece", "Attack on Titan", "Naruto", "Demon Slayer", "My Hero Academia", "Dr. Stone", "Black Clover", "Re:Zero", "Jujutsu Kaisen", "Death Note", "Dragon Ball", "Hunter x Hunter", "One Punch Man", "Neon Genesis Evangelion", "Sword Art Online", "Bleach", "Fullmetal Alchemist", "Haikyuu!!", "JoJo's Bizarre Adventure", "Tokyo Revengers"]},
]

# Parametros usuario
# [Netflix, Netflix, Disney Plus, Primevideo, Crunchyroll, HBO Max, HBO Max, Disney Plus, Peacock, Crunchyroll, Netflix]
contenido_deseado = ["Better Call Saul", "Black Mirror", "Ironman", "The Boys", "One Piece", "The Sopranos", "Westworld","Guardians of the Galaxy", "The Office", "Neon Genesis Evangelion", "Dark", "Los Simpson"]
presupuesto = 15


# Función de evaluación (fitness function)
def evaluate(individual):
    selected_plataformas = [plataformas[i] for i, bit in enumerate(individual) if bit]
    total_cost = sum(plataforma["precio"] for plataforma in selected_plataformas)
    total_content = sum(len(set(plataforma["contenido"]) & set(contenido_deseado)) for plataforma in selected_plataformas)

    if total_cost <= presupuesto:
        if any(len(set(plataforma["contenido"]) & set(contenido_deseado)) == 0 for plataforma in selected_plataformas):
            return 0,
        return total_content,

    return 0,

In [ ]:
# Función para evolucionar la población
def evolve(num_generations, population_size, cxpb, mutpb, tournsize):
    # Inicializar la población
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(plataformas))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.6)
    toolbox.register("select", tools.selTournament, tournsize=tournsize)
    toolbox.register("evaluate", evaluate)

    # Inicializar la población
    population = toolbox.population(n=population_size)

    # Lista para almacenar la aptitud por generación
    fitness_values = []

    # Evolucionar la población
    for gen in range(1, num_generations + 1):
        offspring = algorithms.varAnd(population, toolbox, cxpb=cxpb, mutpb=mutpb)
        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit
        population[:] = offspring
        best_individual_in_gen = tools.selBest(population, k=1)[0]
        fitness_values.append(best_individual_in_gen.fitness.values)


    # Obtener el mejor individuo
    best_individual = tools.selBest(population, k=1)[0]
    selected_plataformas = [plataformas[i] for i, bit in enumerate(best_individual) if bit]

    # Extraer los valores de aptitud de las tuplas
    fitness_values = [fit[0] for fit in fitness_values]

    print("\nMejor combinación de plataformas:")
    for plataforma in selected_plataformas:
        print(f"{plataforma['nombre']} - Contenido: {', '.join(plataforma['contenido'])} - Precio: ${plataforma['precio']}")

    print("Puntaje de contenido total:", sum(len(set(plataforma["contenido"]) & set(contenido_deseado)) for plataforma in selected_plataformas))
    print("Costo total: $", sum(plataforma["precio"] for plataforma in selected_plataformas))

    generations = list(range(1, num_generations + 1))
    plt.plot(generations, fitness_values)
    plt.xlabel('Número de Generación')
    plt.ylabel('Aptitud')
    plt.title('Evolución de la Aptitud')
    plt.grid(True)
    plt.show()



# Widgets y configuraciones
num_generations_widget = widgets.IntSlider(value=200, min=1, max=1000, description='Generations:')
population_size_widget = widgets.IntSlider(value=800, min=10, max=2000, description='Population Size:')
cxpb_widget = widgets.FloatSlider(value=0.7, min=0, max=1, step=0.01, description='Crossover Probability:')
mutpb_widget = widgets.FloatSlider(value=0.2, min=0, max=1, step=0.01, description='Mutation Probability:')
tournsize_widget = widgets.IntSlider(value=3, min=1, max=10, description='Tournament Size:')
iter_widget = widgets.IntSlider(value=2, min=1, max=20, description='Iteration Size:')
play_button = widgets.Button(description="Play")

# Interfaz gráfica para configurar los parámetros
interface = widgets.VBox([num_generations_widget, population_size_widget, cxpb_widget, mutpb_widget, tournsize_widget, iter_widget, play_button])

# Función de callback para evolucionar la población cuando se hace clic en el botón
def on_play_button_click(b):
    for _ in range(iter_widget.value):
      num_generations = num_generations_widget.value
      population_size = population_size_widget.value
      cxpb = cxpb_widget.value
      mutpb = mutpb_widget.value
      tournsize = tournsize_widget.value
      evolve(num_generations, population_size, cxpb, mutpb, tournsize)

# Configurar la función de callback para el botón
play_button.on_click(on_play_button_click)

# Mostrar la interfaz gráfica
display(interface)